In [1]:
!pip install realclearpolitics

In [2]:
!rcp https://www.realclearpolitics.com/epolls/2020/president/tx/texas_trump_vs_biden-6818.html

Downloading: texas_trump_vs_biden-6818.csv
CSV created.


In [3]:
!ls -lh

total 36K
-rw-r--r--. 1 jovyan users  26K Nov 12 06:14 polling_evaluation.ipynb
-rw-r--r--. 1 jovyan  1000  318 Nov 12 03:58 README.md
drwxr-xr-x. 2 jovyan users    6 Nov 12 04:21 spark-warehouse
-rw-r--r--. 1 jovyan users 2.5K Nov 12 06:14 texas_trump_vs_biden-6818.csv


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = (
    SparkSession.builder
    .master("local")
    .appName("polling_error")
    #.config("spark.some.config.option", "some-value")
    .getOrCreate()
)

In [6]:
df = spark.read.csv('texas_trump_vs_biden-6818.csv',
                    header=True,
                    inferSchema=True,
                    enforceSchema=False,
                    mode='FAILFAST')

In [7]:
df = (
    df.withColumnRenamed('Poll', 'poll')
      .withColumnRenamed('Date', 'date_range')
      .withColumnRenamed('Sample', 'sample_size')
      .withColumnRenamed('MoE', 'margin_of_error')
      .withColumnRenamed('Trump (R)', 'trump')
      .withColumnRenamed('Biden (D)', 'biden')
      .withColumnRenamed('Spread', 'spread')
)

In [8]:
df.createOrReplaceTempView('tx_pres_polls_raw')

In [9]:
spark.sql('''
DESCRIBE tx_pres_polls_raw
''').show(truncate=False)

+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|poll           |string   |null   |
|date_range     |string   |null   |
|sample_size    |string   |null   |
|margin_of_error|string   |null   |
|trump          |double   |null   |
|biden          |double   |null   |
|spread         |string   |null   |
+---------------+---------+-------+



In [10]:
spark.sql('''
    SELECT *
    FROM tx_pres_polls_raw
    LIMIT 10
''').show(truncate=False)

+----------------------------------+-------------+-----------+---------------+-----+-----+----------+
|poll                              |date_range   |sample_size|margin_of_error|trump|biden|spread    |
+----------------------------------+-------------+-----------+---------------+-----+-----+----------+
|Final Results                     |--           |--         |--             |52.2 |46.4 |Trump +5.8|
|RCP Average                       |10/20 - 10/31|--         |--             |47.8 |46.5 |Trump +1.3|
|EmersonEmerson                    |10/29 - 10/31|763 LV     |3.5            |49.0 |49.0 |Tie       |
|UMass Lowell*UMass Lowell*        |10/20 - 10/26|873 LV     |4.2            |48.0 |47.0 |Trump +1  |
|NY Times/Siena*NYT/Siena*         |10/20 - 10/25|802 LV     |3.8            |47.0 |43.0 |Trump +4  |
|QuinnipiacQuinnipiac              |10/16 - 10/19|1145 LV    |2.9            |47.0 |47.0 |Tie       |
|University of HoustonU. of Houston|10/13 - 10/20|1000 LV    |3.1            |50.0

In [11]:
spark.sql('''
    SELECT poll,
           '2020-11-03' as election_date,
           trump,
           biden,
           lower(split(spread, ' ', -1)[0]) as winner,
           CASE WHEN split(spread, ' ', -1)[0] = 'Tie'
                THEN 0.0
                ELSE cast(split(spread, ' ', -1)[1] as double) END AS spread
    FROM tx_pres_polls_raw
    WHERE poll = 'Final Results'
''').createOrReplaceTempView('tx_pres_results')

In [12]:
spark.sql('''
    DESCRIBE tx_pres_results
''').show()
spark.sql('''
    SELECT *
    FROM tx_pres_results
    LIMIT 10
''').show(truncate=False)

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|         poll|   string|   null|
|election_date|   string|   null|
|        trump|   double|   null|
|        biden|   double|   null|
|       winner|   string|   null|
|       spread|   double|   null|
+-------------+---------+-------+

+-------------+-------------+-----+-----+------+------+
|poll         |election_date|trump|biden|winner|spread|
+-------------+-------------+-----+-----+------+------+
|Final Results|2020-11-03   |52.2 |46.4 |trump |5.8   |
+-------------+-------------+-----+-----+------+------+



In [13]:
spark.sql('''
    SELECT poll, 
           date_range,
           cast(split(sample_size, ' ', -1)[0] as int) as sample_size,
           cast(margin_of_error as double) as margin_of_error,
           trump,
           biden,
           lower(split(spread, ' ', -1)[0]) as winner,
           split(spread, ' ', -1)[0] = 'Trump' as is_winner_correct,
           CASE WHEN split(spread, ' ', -1)[0] = 'Tie'
                THEN 0.0
                ELSE cast(split(spread, ' ', -1)[1] as double) END AS spread
    FROM tx_pres_polls_raw
    WHERE poll != 'Final Results'
''').createOrReplaceTempView('tx_pres_polls_stage1')

In [14]:
spark.sql('''
    DESCRIBE tx_pres_polls_stage1
''').show()
spark.sql('''
    SELECT *
    FROM tx_pres_polls_stage1
    LIMIT 10
''').show()

+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|             poll|   string|   null|
|       date_range|   string|   null|
|      sample_size|      int|   null|
|  margin_of_error|   double|   null|
|            trump|   double|   null|
|            biden|   double|   null|
|           winner|   string|   null|
|is_winner_correct|  boolean|   null|
|           spread|   double|   null|
+-----------------+---------+-------+

+--------------------+-------------+-----------+---------------+-----+-----+------+-----------------+------+
|                poll|   date_range|sample_size|margin_of_error|trump|biden|winner|is_winner_correct|spread|
+--------------------+-------------+-----------+---------------+-----+-----+------+-----------------+------+
|         RCP Average|10/20 - 10/31|       null|           null| 47.8| 46.5| trump|             true|   1.3|
|      EmersonEmerson|10/29 - 10/31|        763|            3.5| 49.0

In [15]:
spark.sql('''
    SELECT *,
           (SELECT spread FROM tx_pres_results LIMIT 1) as results_spread,
           round(spread - (SELECT spread FROM tx_pres_results LIMIT 1), 1) as spread_diff
    FROM tx_pres_polls_stage1
''').createOrReplaceTempView('tx_pres_polls')

In [16]:
spark.sql('''
    DESCRIBE tx_pres_polls
''').show()
spark.sql('''
    SELECT *
    FROM tx_pres_polls
    LIMIT 10
''').show()

+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|             poll|   string|   null|
|       date_range|   string|   null|
|      sample_size|      int|   null|
|  margin_of_error|   double|   null|
|            trump|   double|   null|
|            biden|   double|   null|
|           winner|   string|   null|
|is_winner_correct|  boolean|   null|
|           spread|   double|   null|
|   results_spread|   double|   null|
|      spread_diff|   double|   null|
+-----------------+---------+-------+

+--------------------+-------------+-----------+---------------+-----+-----+------+-----------------+------+--------------+-----------+
|                poll|   date_range|sample_size|margin_of_error|trump|biden|winner|is_winner_correct|spread|results_spread|spread_diff|
+--------------------+-------------+-----------+---------------+-----+-----+------+-----------------+------+--------------+-----------+
|         RCP Average

In [17]:
print('top 5 polls\n'
      '===========')
spark.sql('''
    SELECT *
    FROM tx_pres_polls
    WHERE is_winner_correct = true
    ORDER BY abs(spread_diff) ASC
    LIMIT 5
''').toPandas()

top 5 polls


,poll,date_range,sample_size,margin_of_error,trump,biden,winner,is_winner_correct,spread,results_spread,spread_diff
0,Trafalgar Group (R)*Trafalgar (R)*,8/1 - 8/5,1015,3.0,49.0,43.0,trump,True,6.0,5.8,0.2
1,EmersonEmerson,5/8 - 5/10,800,3.4,47.0,41.0,trump,True,6.0,5.8,0.2
2,UT/Texas Tribune*UT/Texas Tribune*,9/25 - 10/4,908,4.1,50.0,45.0,trump,True,5.0,5.8,-0.8
3,University of HoustonU. of Houston,10/13 - 10/20,1000,3.1,50.0,45.0,trump,True,5.0,5.8,-0.8
4,Texas LyceumTexas Lyceum,1/10 - 1/19,520,4.3,51.0,46.0,trump,True,5.0,5.8,-0.8


In [18]:
print('bottom 5 polls\n'
      '===========')
spark.sql('''
    SELECT *
    FROM tx_pres_polls
    WHERE is_winner_correct = false
    ORDER BY abs(spread_diff) DESC
    LIMIT 5
''').toPandas()

bottom 5 polls


,poll,date_range,sample_size,margin_of_error,trump,biden,winner,is_winner_correct,spread,results_spread,spread_diff
0,Dallas Morning NewsDMN,4/18 - 4/27,1183,2.9,43.0,43.0,tie,False,0.0,5.8,-5.8
1,QuinnipiacQuinnipiac,10/16 - 10/19,1145,2.9,47.0,47.0,tie,False,0.0,5.8,-5.8
2,EmersonEmerson,10/29 - 10/31,763,3.5,49.0,49.0,tie,False,0.0,5.8,-5.8
3,CNNCNN,2/22 - 2/26,1003,3.0,47.0,48.0,biden,False,1.0,5.8,-4.8
4,FOX NewsFOX News,6/20 - 6/23,1001,3.0,44.0,45.0,biden,False,1.0,5.8,-4.8
